In [10]:
def allEntities(data_map, date_range):
    all_entities = set()
    for date in date_range:
        all_entities = all_entities.union(set(data_map[date].keys()))
    return all_entities

def standardizeEntities(data_map, all_entities):
    new_data_map = {}
    for k, v in data_map.items():
        date_dict = {}
        for ent in all_entities:
            if ent in v.keys():
                date_dict[ent] = v[ent]
            else:
                date_dict[ent] = 0
        new_data_map[k] = date_dict
    return new_data_map

def initEntityMap(all_entities):
    entity_map = {}
    for ent in all_entities:
        entity_map[ent] = {
            "deep_red":0,
            "light_red":0,
            "light_green":0,
            "deep_green":0,
        }
    return entity_map

def assignEntToBucket(entDeviation):
    if entDeviation < 0:
        raise Exception("Entity Deviation cannot be negative")
    if entDeviation == 0:
        return "deep_red"
    elif entDeviation > 1.10:
        return "light_red"
    elif entDeviation > 0 and entDeviation < .80:
        return "light_green"
    else:
        return "deep_green"
    
def outputChart(entity_map, fname="chart.html"):
    tot_str = f"["
    for ent in entity_map:
        tot_str += '{{ category: "{}",'.format(ent)
        tot_str += 'negative2: -{},'.format( entity_map[ent]["deep_red"] )    
        tot_str += 'negative1: -{},'.format( entity_map[ent]["light_red"] )
        tot_str += 'positive1: {},'.format( entity_map[ent]["light_green"] )
        tot_str += 'positive2: {},'.format( entity_map[ent]["deep_green"] )    
        tot_str += '}},'
    tot_str += "]"   
    
    chart_code = '''
        <style>
        #chartdiv {{
          width: 100%;
          height: 500px;
        }}
        </style>

        <!-- Resources -->
        <script src="https://cdn.amcharts.com/lib/5/index.js"></script>
        <script src="https://cdn.amcharts.com/lib/5/xy.js"></script>
        <script src="https://cdn.amcharts.com/lib/5/themes/Animated.js"></script>

        <!-- Chart code -->
        <script>
        am5.ready(function() {{

        // Create root element
        // https://www.amcharts.com/docs/v5/getting-started/#Root_element
        var root = am5.Root.new("chartdiv");

        // Set themes
        // https://www.amcharts.com/docs/v5/concepts/themes/
        root.setThemes([
          am5themes_Animated.new(root)
        ]);

        // Create chart
        // https://www.amcharts.com/docs/v5/charts/xy-chart/
        var chart = root.container.children.push(
          am5xy.XYChart.new(root, {{
            panX: false,
            panY: false,
            wheelX: "panX",
            wheelY: "zoomX",
            layout: root.horizontalLayout,
            arrangeTooltips: false
          }})
        );

        // Use only absolute numbers
        root.numberFormatter.set("numberFormat", "#.#s'%");

        // Add legend
        // https://www.amcharts.com/docs/v5/charts/xy-chart/legend-xy-series/
        var legend = chart.children.push(
          am5.Legend.new(root, {{
            centerX: am5.p50,
            x: am5.p50
          }})
        );

        // Data
        var data = {0};

        // Create axes
        // https://www.amcharts.com/docs/v5/charts/xy-chart/axes/
        var yAxis = chart.yAxes.push(
          am5xy.CategoryAxis.new(root, {{
            categoryField: "category",
            renderer: am5xy.AxisRendererY.new(root, {{
              inversed: true,
              cellStartLocation: 0.1,
              cellEndLocation: 0.9
            }})
          }})
        );

        yAxis.data.setAll(data);

        var xAxis = chart.xAxes.push(
          am5xy.ValueAxis.new(root, {{
            calculateTotals: true,
            min: -100,
            max: 100,
            renderer: am5xy.AxisRendererX.new(root, {{
              minGridDistance: 50
            }})
          }})
        );

        var xRenderer = yAxis.get("renderer");
        xRenderer.axisFills.template.setAll({{
          fill: am5.color(0x000000),
          fillOpacity: 0.05,
          visible: true
        }});

        // Add series
        // https://www.amcharts.com/docs/v5/charts/xy-chart/series/
        function createSeries(field, name, color) {{
          var series = chart.series.push(
            am5xy.ColumnSeries.new(root, {{
              xAxis: xAxis,
              yAxis: yAxis,
              name: name,
              valueXField: field,
              valueXShow: "valueXTotalPercent",
              categoryYField: "category",
              sequencedInterpolation: true,
              stacked: true,
              fill: color,
              stroke: color,
              calculateAggregates: true
            }})
          );

          series.columns.template.setAll({{
            height: am5.p100
          }});

          series.bullets.push(function(root, series) {{
            return am5.Bullet.new(root, {{
              locationX: 0.5,
              locationY: 0.5,
              sprite: am5.Label.new(root, {{
                fill: am5.color(0xffffff),
                centerX: am5.p50,
                centerY: am5.p50,
                text: "{{valueX}}",
                populateText: true,
                oversizedBehavior: "hide"
              }})
            }});
          }});

          series.data.setAll(data);
          series.appear();

          return series;
        }}

        var positiveColor = root.interfaceColors.get("positive");
        var negativeColor = root.interfaceColors.get("negative");

        createSeries("negative2", "Not Present", am5.Color.lighten(negativeColor, 0.5));
        createSeries("negative1", "More than 110 deviation%", negativeColor);
        createSeries("positive1", "0%-80% deviation", am5.Color.lighten(positiveColor, 0.5));
        createSeries("positive2", "80%-100% deviation", positiveColor);

        // Add legend
        // https://www.amcharts.com/docs/v5/charts/xy-chart/legend-xy-series/
        var legend = chart.children.push(
          am5.Legend.new(root, {{
            centerY: am5.p50,
            y: am5.p50,
            layout: root.verticalLayout,
            marginLeft: 50
          }})
        );

        legend.data.setAll(chart.series.values);

        // Make stuff animate on load
        // https://www.amcharts.com/docs/v5/concepts/animations/
        chart.appear(1000, 100);

        }}); // end am5.ready()
        </script>

        <!-- HTML -->
        <div id="chartdiv"></div>
        '''
    
    chart_html_file = open(fname, "w")
    chart_html_file.write(chart_code.format(tot_str).replace("}}", "}").replace("{{", "{") )
    chart_html_file.close()    

In [17]:
data_map = {
    '2022-01-01':{
        # entity - entDeviation map
        "ent1": .40,
        "ent2": 1.12,
        "ent3": .90
    },
    '2022-01-02':{
        "ent1": .40,
        "ent3": .90
    },
    '2022-01-03':{
        "ent2": .80,
        "ent3": .90
    },
}



time_range   = ['2022-01-01', '2022-01-02', '2022-01-03']
#time_range   = ['2022-01-01']
all_entities = allEntities(data_map, time_range)
stand_data_map = standardizeEntities(data_map, all_entities)
entity_map   = initEntityMap(all_entities)

# for now it's going to be linear, we are going to make it exponential later
time_weight = 0
for t in time_range:
    time_weight += 1
    print("Processing ", t)
    for k, v in stand_data_map[t].items():
        targ_bucket = assignEntToBucket(v)
        entity_map[k][targ_bucket] += time_weight

denom = (time_weight*(1+time_weight))/2
for k, v in entity_map.items():
    for k1, v1 in v.items():
        v[k1] = v1/denom
        v[k1] *= 100
outputChart(entity_map)

Processing  2022-01-01
Processing  2022-01-02
Processing  2022-01-03
